In [6]:
from tqdm import tqdm
import os
os.chdir('/Users/grace/Cornell-Conversational-Analysis-Toolkit/')
from convokit import Corpus, User, Utterance

In [7]:
os.chdir('/Users/grace/Cornell-Conversational-Analysis-Toolkit/datasets/')
corpus = Corpus(filename='persuasion_corpus/corpus')
persuasioncorpus = corpus #Have to rename corpus as persuasion corpus so as not to mess up internal functions for later

In [8]:
corpus.print_summary_stats()

Number of Users: 1285
Number of Utterances: 20932
Number of Conversations: 1017


In [4]:
dir(corpus)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_user_data',
 '_merge_utterances',
 '_update_corpus_user_data',
 'add_meta',
 'add_utterances',
 'all_users',
 'conversations',
 'dump',
 'dump_helper_bin',
 'filter_utterances_by',
 'get_conversation',
 'get_conversation_ids',
 'get_meta',
 'get_user',
 'get_usernames',
 'get_utterance',
 'get_utterance_ids',
 'iter_conversations',
 'iter_users',
 'iter_utterances',
 'iterate_by',
 'merge',
 'meta',
 'meta_index',
 'original_corpus_path',
 'pairwise_exchanges',
 'print_summary_stats',
 'speaking_pairs',
 'update_users_data',
 'utterance_threads',
 'utterances',
 'version']

## Average Number of Questions Per Conversation

In [5]:
import re
import numpy as np

In [6]:
np.mean(np.asarray(questions)[np.asarray(corpus.get_conversation(0)._utterance_ids)])

NameError: name 'questions' is not defined

In [7]:
questions = []
allutterids = corpus.get_utterance_ids()
for i in list(range(0, len(allutterids))):
    utter_id = allutterids[i]
    text = corpus.get_utterance(utter_id).text
    nquestions = len(re.findall(r'\?+', text))
    questions.append(nquestions) #gives number of questions in each utterance


In [8]:
allconvoids = corpus.get_conversation_ids()
for i in list(range(0, len(allconvoids))):
    convo_id = allconvoids[i]
    convo_utters = corpus.get_conversation(convo_id)._utterance_ids
    avgquestion = np.mean(np.asarray(questions)[np.asarray(convo_utters)])
    corpus.get_conversation(convo_id)._meta["Avg Questions"] = avgquestion
    

In [9]:
corpus.get_conversation(0)

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x1a207bc390>, '_id': 0, '_utterance_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], '_usernames': None, '_meta': {'Avg Questions': 0.38095238095238093}})

Things to do now:
1. Put this function into the avgQuestions.py
2. Test calling on this transformer and seeing if it gives results correctly

In [10]:
from convokit import avgQuestions

In [11]:
avgQuestions.AvgQuestions

convokit.avgQuestions.avgQuestions.AvgQuestions

In [12]:
nQuestions = avgQuestions.AvgQuestions()
corpus2 = nQuestions.transform(persuasioncorpus)

In [13]:
corpus2.get_conversation(0) #Matches results!

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x1a207bc390>, '_id': 0, '_utterance_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], '_usernames': None, '_meta': {'Avg Questions': 0.38095238095238093, 'AvgQuestions': 0.38095238095238093}})

## Creating Hesitancy Measure

In [41]:
pause_words = ['um', 'umm', 'ummm', 'uh', 'uhh', 'uhhh', 'hm', 'hmm', 'hmmm', 'er', 'err', 'uh huh', 'huh',
               'mhm', 'mhmm', 'erm', '...', 'ah', 'ahh', 'ahem', 'eh', 'ehh', 'ehhh', 'meh']
hesitant_words = ['maybe', 'not',  'sure', 'unsure', 'probably', 'well', 'okay', 'like', 'actually', 
                   'basically', 'seriously', 'totally', 'literally', 'know', 'mean', 
                   'guess', 'suppose', 'but', 'something', 'so', 'wow', 'just', 'really', 
                  'later', 'wait', 'future', 'almost', 'slightly', 'perhaps', 'somehow', 
                 'sort', 'kind', 'little', 'somewhat', 'hey', 'alas', 'see', 'sounds', 'ok',
                 'roughly', 'why', 'how', 'yep', 'yup', 'may', 'possibly', 'might', 'could', 'doubt',
                 'skeptical', 'don\'t', 'won\'t', 'nah']
text = "trial string something random*#&$ dkj okay@ now what? hey sounds literally"
textcleaned = "".join(c for c in text if c not in ('!','.',':', '?', '\'', ',', '\"', '@', '#', '$', '%', 
                                                       '^', '&', '*', '(', ')', '-', '~', '`', '_', '+', '=', 
                                                       '>', '<', '[', ']', '{', '}'))
textlist = textcleaned.split()
textsubset = [i for i in textlist if i in hesitant_words]
len(textsubset)

5

In [42]:
pause = []
hesitancy = []
allutterids = corpus.get_utterance_ids()
for i in list(range(0, len(allutterids))):
    utter_id = allutterids[i]
    text = corpus.get_utterance(utter_id).text
    textcleaned = "".join(c for c in text if c not in ('!','.',':', '?', '\'', ',', '\"', '@', '#', '$', '%', 
                                                       '^', '&', '*', '(', ')', '-', '~', '`', '_', '+', '=', 
                                                       '>', '<', '[', ']', '{', '}'))
    textlist = textcleaned.split()
    npause = len([i for i in textlist if i in pause_words])
    nhesitant = len([i for i in textlist if i in hesitant_words])
    pause.append(npause) #gives number of pause words in each utterance
    hesitancy.append(nhesitant) #gives number of hesitant words in each utterance
    corpus.get_utterance(utter_id).meta['Pause'] = npause
    corpus.get_utterance(utter_id).meta['Hesitancy'] = nhesitant

In [43]:
import numpy as np
print(np.mean(np.asarray(hesitancy)))
print(np.mean(np.asarray(pause)))
np.sum(np.asarray(hesitancy)[0:19])

0.8692432639021593
0.0009077011274603477


15

In [45]:
corpus.get_utterance(0)

Utterance({'id': 0, 'user': User([('name', 'A3A07QA5U733HQ')]), 'root': 0, 'reply_to': None, 'timestamp': None, 'text': 'Good morning. How are you doing today?', 'meta': {'role': 0, 'turn_index': 0, 'dialogue_id': '20180904-045349_715_live', 'er_label_1': 'greeting', 'ee_label_1': None, 'er_label_2': None, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'Pause': 0, 'Hesitancy': 0}})

In [46]:
allconvoids = corpus.get_conversation_ids()
for i in list(range(0, len(allconvoids))):
    convo_id = allconvoids[i]
    convo_utters = corpus.get_conversation(convo_id)._utterance_ids
    avgpause = np.mean(np.asarray(pause)[np.asarray(convo_utters)])
    avghesitancy = np.mean(np.asarray(hesitancy)[np.asarray(convo_utters)])
    corpus.get_conversation(convo_id)._meta["Avg Pause"] = avgpause
    corpus.get_conversation(convo_id)._meta["Avg Hesitancy"] = avghesitancy
    

In [47]:
corpus.get_conversation(0)

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x10a07bd68>, '_id': 0, '_utterance_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], '_usernames': None, '_meta': {'Avg Pause': 0.0, 'Avg Hesitancy': 0.8095238095238095}})

In [35]:
from convokit import hesitancy

In [36]:
hesitancy.hesitancy

convokit.hesitancy.hesitancy.hesitancy

In [37]:
h = hesitancy.hesitancy()
corpus3 = h.transform(persuasioncorpus)

In [39]:
corpus3.get_utterance(0) #result matches!

Utterance({'id': 0, 'user': User([('name', 'A3A07QA5U733HQ')]), 'root': 0, 'reply_to': None, 'timestamp': None, 'text': 'Good morning. How are you doing today?', 'meta': {'role': 0, 'turn_index': 0, 'dialogue_id': '20180904-045349_715_live', 'er_label_1': 'greeting', 'ee_label_1': None, 'er_label_2': None, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'Pause': 0, 'Hesitancy': 0}})

In [40]:
corpus3.get_conversation(0) #result matches!

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x10a07bd68>, '_id': 0, '_utterance_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], '_usernames': None, '_meta': {'Avg Pause': 0.0, 'Avg Hesitancy': 0.8095238095238095}})